# Samples by year and by continent

## Prepare the CSV docs

Convert the relevant sheet from Reich's excel doc into a csv

In [6]:
!in2csv --sheet 'raw.Feb.23.2018' ref/2_23_2018\ Growth\ in\ aDNA\ sample\ size\ Ewen\ Revision.xls > data/raw_sites.csv

/Users/c.ryan/anaconda/lib/python2.7/site-packages/agate/utils.py:275: UnnamedColumnWarning: Column 11 has no name. Using "l".
/Users/c.ryan/anaconda/lib/python2.7/site-packages/agate/utils.py:275: UnnamedColumnWarning: Column 12 has no name. Using "m".


Print out a list of the column names

In [8]:
!csvcut -n data/raw_sites.csv

  1: Sample ID
  2: Master ID
  3: Skeletal codes
  4: Data type
  5: Publication
  6:   Average of 95.4% date range in calBP (defined as 1950 CE)  
  7: Label
  8: Location
  9: Country
 10: Year published
 11: Coverage
 12: l
 13: m


Extract just the country and the year columns into a new csv

In [10]:
!csvcut -c 9,10 data/raw_sites.csv | csvlook | head

| Country         | Year published |
| --------------- | -------------- |
| Russia          |          2,010 |
| Russia          |          2,012 |
| Luxembourg      |          2,018 |
| Luxembourg      |          2,014 |
| Sweden          |          2,015 |
| Sweden          |          2,014 |
| Germany         |          2,017 |
| Germany         |          2,014 |


In [11]:
!csvcut -c 9,10 data/raw_sites.csv > data/sites.csv

Load the (countries shapefile)[http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip] from Natural Earth Data into QGIS and export the layer as a csv to get a list of countries and continents.

Print out a list of the column headers

In [14]:
!in2csv -e iso-8859-1 ref/natural-earth-data-countries.csv | csvcut -n

  1: scalerank
  2: featurecla
  3: LABELRANK
  4: SOVEREIGNT
  5: SOV_A3
  6: ADM0_DIF
  7: LEVEL
  8: TYPE
  9: ADMIN
 10: ADM0_A3
 11: GEOU_DIF
 12: GEOUNIT
 13: GU_A3
 14: SU_DIF
 15: SUBUNIT
 16: SU_A3
 17: BRK_DIFF
 18: NAME
 19: NAME_LONG
 20: BRK_A3
 21: BRK_NAME
 22: BRK_GROUP
 23: ABBREV
 24: POSTAL
 25: FORMAL_EN
 26: FORMAL_FR
 27: NAME_CIAWF
 28: NOTE_ADM0
 29: NOTE_BRK
 30: NAME_SORT
 31: NAME_ALT
 32: MAPCOLOR7
 33: MAPCOLOR8
 34: MAPCOLOR9
 35: MAPCOLOR13
 36: POP_EST
 37: POP_RANK
 38: GDP_MD_EST
 39: POP_YEAR
 40: LASTCENSUS
 41: GDP_YEAR
 42: ECONOMY
 43: INCOME_GRP
 44: WIKIPEDIA
 45: FIPS_10_
 46: ISO_A2
 47: ISO_A3
 48: ISO_A3_EH
 49: ISO_N3
 50: UN_A3
 51: WB_A2
 52: WB_A3
 53: WOE_ID
 54: WOE_ID_EH
 55: WOE_NOTE
 56: ADM0_A3_IS
 57: ADM0_A3_US
 58: ADM0_A3_UN
 59: ADM0_A3_WB
 60: CONTINENT
 61: REGION_UN
 62: SUBREGION
 63: REGION_WB
 64: NAME_LEN
 65: LONG_LEN
 66: ABBREV_LEN
 67: TINY
 68: HOMEPART
 69: MIN_ZOOM
 70: MIN_LABEL
 71: MAX_LABEL


Extract just the country name an continent into a new csv file

In [15]:
!in2csv -e iso-8859-1 ref/natural-earth-data-countries.csv | csvcut -c 9,60 > data/continents.csv

## Load the csvs into DataFrames

Also rename the columns so they can be easily merged

In [16]:
import pandas as pd

In [17]:
sites = pd.read_csv('data/sites.csv')

In [18]:
sites.head()

,Country,Year published
0,Russia,2010.0
1,Russia,2012.0
2,Luxembourg,2018.0
3,Luxembourg,2014.0
4,Sweden,2015.0


In [24]:
sites.rename(index=str, columns={"Country":"country","Year published":"year"}, inplace=True)

In [25]:
sites.head()

,country,year
0,Russia,2010.0
1,Russia,2012.0
2,Luxembourg,2018.0
3,Luxembourg,2014.0
4,Sweden,2015.0


In [21]:
continents = pd.read_csv('data/continents.csv')

In [22]:
continents.head()

,ADMIN,CONTINENT
0,Afghanistan,Asia
1,Angola,Africa
2,Albania,Europe
3,United Arab Emirates,Asia
4,Argentina,South America


In [26]:
continents.rename(index=str, columns={"ADMIN":"country", "CONTINENT":"continent"}, inplace=True)

In [27]:
continents.head()

,country,continent
0,Afghanistan,Asia
1,Angola,Africa
2,Albania,Europe
3,United Arab Emirates,Asia
4,Argentina,South America


In [28]:
test = pd.merge(sites, continents, on="country", how="left")

In [70]:
test.head()

,country,year,continent
0,Russia,2010.0,Europe
1,Russia,2012.0,Europe
2,Luxembourg,2018.0,Europe
3,Luxembourg,2014.0,Europe
4,Sweden,2015.0,Europe


## Merge the DataFrames

Make a test merge, so we can see how many countries don't match up

In [40]:
test[test["continent"].isnull()]["country"].unique()

array(['Tanzania', 'Czech Republic', 'Great Britain', 'USA',
       'Canary Islands', 'Serbia', 'The Netherlands', '..', 'Tonga',
       'Bahamas', nan], dtype=object)

Rename 'Tanzania', 'Czech Republic', 'Great Britain', 'USA', 'Canary Islands', 'Serbia', 'The Netherlands', 'Tonga' and 'Bahamas' so they are consistent across both DataFrames

In [46]:
sites.replace(to_replace="Tanzania", value="United Republic of Tanzania", inplace=True)

In [47]:
sites.replace(to_replace="Czech Republic", value="Czechia", inplace=True)

In [48]:
sites.replace(to_replace="Great Britain", value="United Kingdom", inplace=True)

In [49]:
sites.replace(to_replace="USA", value="United States of America", inplace=True)

In [50]:
sites.replace(to_replace="Canary Islands", value="Spain", inplace=True)

In [51]:
sites.replace(to_replace="Serbia", value="Republic of Serbia", inplace=True)

In [52]:
sites.replace(to_replace="The Netherlands", value="Netherlands", inplace=True)

In [53]:
sites.replace(to_replace="Tonga", value="Fiji", inplace=True)

In [54]:
sites.replace(to_replace="Bahamas", value="The Bahamas", inplace=True)

There are four countries listed as '..'

In [61]:
sites[sites["country"] == ".."]

,country,year
1177,..,2014.0
1194,..,2014.0
1196,..,2014.0
1348,..,2014.0


Merge the amended DataFrames with an inner join so that records that don't match will be removed

In [62]:
merge = pd.merge(sites, continents, on="country", how="inner")

In [66]:
merge.head()

,country,year,continent
0,Russia,2010.0,Europe
1,Russia,2012.0,Europe
2,Russia,2015.0,Europe
3,Russia,2016.0,Europe
4,Russia,2016.0,Europe


## Count

Group by year and by continent

In [79]:
by_year = merge.groupby(["year", "continent"])

In [81]:
by_year.count().head(100)

country
year   continent             
2010.0 Europe               4
       North America        1
2011.0 Oceania              1
2012.0 Europe               1
2013.0 Europe               4
2014.0 Europe              31
       North America        1
       South America        2
2015.0 Africa               1
       Asia                37
       Europe             239
       North America       11
       South America       13
2016.0 Asia                73
       Europe              65
       Oceania              4
2017.0 Africa              30
       Asia                12
       Europe             197
       North America        1
2018.0 Asia                 1
       Europe             666
       North America        2

Count each row and reset the index so that a DataFrame can be printed to a csv

In [94]:
output = by_year.count().reset_index()

In [95]:
output.to_csv("output.csv")